## Setup and Imports

In [ ]:
!pip install -U -q timm transformers

In [ ]:
import io
import os
import zipfile

import torch
from datasets import load_dataset
from PIL import Image
from transformers import AutoProcessor, Gemma3nForConditionalGeneration

from trl import (
    SFTConfig,
    SFTTrainer,
)

## Dataset

In [ ]:
dataset = load_dataset("ariG23498/intersection-dataset")

In [ ]:
def format_intersection_data(samples: dict) -> dict[str, list]:
    """Format intersection dataset to match expected message format"""
    formatted_samples = {"messages": []}
    for idx in range(len(samples["image"])):
        image = samples["image"][idx].convert("RGB")
        label = str(samples["label"][idx])

        message = [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": "You are an assistant with great geometry skills.",
                    }
                ],
            },
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {
                        "type": "text",
                        "text": "How many intersection points are there in the image?",
                    },
                ],
            },
            {"role": "assistant", "content": [{"type": "text", "text": label}]},
        ]
        formatted_samples["messages"].append(message)
    return formatted_samples

In [ ]:
dataset = dataset.map(format_intersection_data, batched=True, batch_size=4, num_proc=4)

model = Gemma3nForConditionalGeneration.from_pretrained(
    "google/gemma-3n-E2B-it", torch_dtype=torch.bfloat16,
)
processor = AutoProcessor.from_pretrained(
    "google/gemma-3n-E2B-it",
)
processor.tokenizer.padding_side = "right"

In [ ]:
def process_vision_info(messages: list[dict]) -> list[Image.Image]:
    image_inputs = []
    for msg in messages:
        content = msg.get("content", [])
        if not isinstance(content, list):
            content = [content]

        for element in content:
            if isinstance(element, dict) and ("image" in element or element.get("type") == "image"):
                if "image" in element:
                    image = element["image"]
                else:
                    image = element
                if image is not None:
                    # Handle dictionary with bytes
                    if isinstance(image, dict) and "bytes" in image:
                        pil_image = Image.open(io.BytesIO(image["bytes"]))
                        image_inputs.append(pil_image.convert("RGB"))
                    # Handle PIL Image objects
                    elif hasattr(image, "convert"):
                        image_inputs.append(image.convert("RGB"))
    return image_inputs

In [1]:
def collate_fn(examples):
        texts = []
        images_list = []

        for example in examples:
            # Apply chat template to get text
            text = processor.apply_chat_template(
                example["messages"], tokenize=False, add_generation_prompt=False
            ).strip()
            texts.append(text)

            # Extract images
            if "images" in example:  # single-image case
                images = [img.convert("RGB") for img in example["images"]]
            else:  # multi-image case or intersection dataset
                images = process_vision_info(example["messages"])
            images_list.append(images)

        # Tokenize the texts and process the images
        batch = processor(
            text=texts, images=images_list, return_tensors="pt", padding=True
        )

        # The labels are the input_ids, and we mask the padding tokens in the loss computation
        labels = batch["input_ids"].clone()

        # Use Gemma3n specific token masking
        labels[labels == processor.tokenizer.pad_token_id] = -100
        if hasattr(processor.tokenizer, 'image_token_id'):
            labels[labels == processor.tokenizer.image_token_id] = -100
        if hasattr(processor.tokenizer, "audio_token_id"):
            labels[labels == processor.tokenizer.audio_token_id] = -100
        if hasattr(processor.tokenizer, 'boi_token_id'):
            labels[labels == processor.tokenizer.boi_token_id] = -100
        if hasattr(processor.tokenizer, 'eoi_token_id'):
            labels[labels == processor.tokenizer.eoi_token_id] = -100


        batch["labels"] = labels
        return batch

## Training

In [ ]:
from peft import LoraConfig
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    target_modules="all-linear",
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_rslora=False,
    use_dora=False,
    modules_to_save=None,
)

In [ ]:
training_args = SFTConfig(
    output_dir="/content/gemma-3n-E2B-it-trl-sft-intersection",
    eval_strategy='no',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    learning_rate=1e-05,
    num_train_epochs=1.0,
    logging_steps=10,
    save_steps=100,
    bf16=True,
    report_to=["wandb"],
    run_name='gemma-3n-E2B-it-trl-sft-intersection',
    dataset_kwargs={'skip_prepare_dataset': True},
    remove_unused_columns=False,
    max_seq_length=None,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"] if training_args.eval_strategy != "no" else None,
    processing_class=processor.tokenizer,
    peft_config=peft_config,
)

In [ ]:
trainer.train()